In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
from mlxtend.preprocessing import TransactionEncoder
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import mlxtend

In [ ]:
# Đọc dữ liệu từ file chipotle.tsv
df = pd.read_csv('chipotle.tsv', sep='\t')

In [ ]:
print(df.info())
print(df.head())

In [ ]:
print(df.choice_description.head(5)) 

In [ ]:
print(df.item_price)

In [ ]:
df.describe(include='all')

In [ ]:
##Kiểm tra missing data bằng trực quan hóa
fig,ax=plt.subplots(figsize=(8,5))
sns.heatmap(df.isna(),cmap='Blues',cbar= False, yticklabels=False);

In [ ]:
# Kiểm tra các giá trị null trong cột 'quantity'
null_quantity_rows = df[df['quantity'].isnull()]
print(null_quantity_rows)

In [ ]:
# Làm sạch dữ liệu, loại bỏ các dòng không có món ăn
cleaned_data = df.dropna(subset=['item_name'])
print(cleaned_data)

In [ ]:
print(df.choice_description.head(5)) 

In [ ]:
# Chỉ thay thế các giá trị null trong cột 'choice_description' thành 'none'
df['choice_description'] = df['choice_description'].where(df['choice_description'].notnull(), 'none')

In [ ]:
# Kiểm tra các hàng có giá trị bằng 0 trong cột 'quantity'
zero_quantity_rows = df[df['quantity'] == 0]

# Đếm số lượng hàng có giá trị bằng 0
zero_quantity_count = zero_quantity_rows.shape[0]

# Hiển thị kết quả
print(f"Số lượng hàng có giá trị bằng 0 trong cột 'quantity': {zero_quantity_count}")
if zero_quantity_count > 0:
    print("Mẫu dữ liệu có giá trị bằng 0 trong cột 'quantity':")
    print(zero_quantity_rows)
else:
    print("Không có giá trị bằng 0 trong cột 'quantity'.")
print(df.info())

In [ ]:
# Kiểm tra lại để đảm bảo không còn giá trị null
print(f"Số lượng giá trị null còn lại trong cột 'choice_description': {df['choice_description'].isnull().sum()}")
print("Một số hàng đầu của cột 'choice_description':")
print(df['choice_description'].head())

In [ ]:
print(df['choice_description'].isnull().sum()) 

In [ ]:
# Kiểm tra các giá trị null trong cột 'item_price'
null_item_price = df[df['item_price'].isnull()]
print(null_item_price)

In [ ]:
# Kiểm tra các hàng có giá trị bằng 0 trong cột 'item_price'
zero_itemprice = df[df['item_price'] == 0]

# Đếm số lượng hàng có giá trị bằng 0
zero_itemprice_count = zero_itemprice.shape[0]

# Hiển thị kết quả
print(f"Số lượng hàng có giá trị bằng 0 trong cột 'item_price': {zero_itemprice_count}")
if zero_itemprice_count > 0:
    print("Mẫu dữ liệu có giá trị bằng 0 trong cột 'quantity':")
    print(zero_itemprice)
else:
    print("Không có giá trị bằng 0 trong cột 'item_price'.")

In [ ]:
df.item_price.dtype

In [ ]:
# Đảm bảo tất cả các giá trị trong cột 'item_price' là chuỗi
df['item_price'] = df['item_price'].astype(str)

# Xử lý NaN bằng cách thay thế bằng giá trị mặc định
df['item_price'] = df['item_price'].replace('NaN', '$0')

# Loại bỏ ký hiệu '$' và chuyển thành kiểu float
df['item_price'] = df['item_price'].str.replace('$', '', regex=False)
df['item_price'] = df['item_price'].astype(float)

# Hiển thị kết quả
print(df['item_price'])

In [ ]:
df.head()

## CHUẨN BỊ DỮ LIỆU

In [ ]:
df['total_price'] = df['quantity']*df['item_price']

In [ ]:
revenue=df['total_price'].sum()
print(revenue)

In [ ]:
# Nhóm dữ liệu theo 'order_id' để tạo danh sách các sản phẩm trong từng đơn hàng
grouped_data = df.groupby('order_id')['item_name'].apply(list).reset_index()
print(grouped_data.head())

In [ ]:
from itertools import combinations
from collections import Counter
import pandas as pd

# Lấy danh sách các sản phẩm theo từng đơn hàng
order_products = df.groupby('order_id')['item_name'].apply(list)

# Tổng hợp doanh thu theo từng cặp sản phẩm
pair_revenue = Counter()
for products in order_products:
    for pair in combinations(sorted(products), 2):
        # Lọc dữ liệu liên quan đến từng cặp sản phẩm
        relevant_rows = df[df['item_name'].isin(pair)]
        # Tính tổng doanh thu cho cặp sản phẩm
        pair_revenue[pair] += relevant_rows['total_price'].sum()

# Chuyển đổi kết quả thành DataFrame
pair_revenue_df = pd.DataFrame(pair_revenue.items(), columns=['pair', 'revenue'])

# Sắp xếp giảm dần theo doanh thu và lấy top 10
top_revenue_pairs = pair_revenue_df.sort_values(by='revenue', ascending=False).head(10)

# Hiển thị kết quả
print(top_revenue_pairs)

In [ ]:
# Vẽ biểu đồ top 10 cặp sản phẩm có doanh thu cao nhất
plt.figure(figsize=(12, 6))
plt.barh(
    [f"{pair[0]} & {pair[1]}" for pair in top_revenue_pairs['pair']], 
    top_revenue_pairs['revenue'], 
    color='skyblue'
)
plt.xlabel("Doanh thu ($)", fontsize=12)
plt.ylabel("Cặp sản phẩm", fontsize=12)
plt.title("Top 10 Cặp Sản Phẩm Có Doanh Thu Cao Nhất", fontsize=14)
plt.gca().invert_yaxis()  # Đảo ngược trục Y để cặp có doanh thu cao nhất ở trên cùng
plt.show()

In [ ]:
# Tổng hợp doanh thu theo từng bộ ba sản phẩm
triplet_revenue = Counter()
for products in order_products:
    for triplet in combinations(sorted(products), 3):  # Xét bộ ba sản phẩm
        # Lọc dữ liệu liên quan đến từng bộ ba sản phẩm
        relevant_rows = df[df['item_name'].isin(triplet)]
        # Tính tổng doanh thu cho bộ ba sản phẩm
        triplet_revenue[triplet] += relevant_rows['total_price'].sum()

# Chuyển đổi kết quả thành DataFrame
triplet_revenue_df = pd.DataFrame(triplet_revenue.items(), columns=['triplet', 'revenue'])

# Sắp xếp giảm dần theo doanh thu và lấy top 10
top_revenue_triplets = triplet_revenue_df.sort_values(by='revenue', ascending=False).head(10)

# Hiển thị kết quả
print(top_revenue_triplets)

# ở 3 cặp sản phẩm, có những cặp có sản phẩm trùng nhau nên không chọn tổng hợp doanh thu từng bộ 3 sp

In [ ]:
##nhóm item_name và quantity + tính tổng
group_d1=df.groupby('item_name')['quantity'].sum()
group_d1.sort_values(ascending= False).head(10)

In [ ]:
##số món hàng được bán ra trong ngày hôm đó
df.item_name.nunique()

In [ ]:
grouped_data.shape

In [ ]:
from mlxtend.preprocessing import TransactionEncoder

# Chuẩn bị dữ liệu dạng danh sách giao dịch
transactions = grouped_data['item_name'].tolist()

print(transactions)

In [ ]:
from mlxtend.preprocessing import TransactionEncoder
import pandas as pd

# Đảm bảo không bỏ sót giá trị thiếu
df = df.dropna(subset=['item_name'])

# Kiểm tra danh sách giao dịch
transactions = grouped_data['item_name'].tolist()
print("Số lượng giao dịch:", len(transactions))
print(transactions[:3])

In [ ]:
print(grouped_data)

In [ ]:
# Mã hóa dữ liệu giao dịch thành ma trận nhị phân
te = TransactionEncoder()
te_ary = te.fit_transform(transactions)
df_matrix= pd.DataFrame(te_ary, columns=te.columns_)

# Kiểm tra ma trận và danh sách sản phẩm
print("Số lượng sản phẩm:", len(te.columns_))
print(df_matrix.head(5))

In [ ]:
from mlxtend.frequent_patterns import apriori

# Áp dụng thuật toán Apriori để tìm các tập phổ biến
frequent_itemsets = apriori(df_matrix, min_support=0.02, use_colnames=True) 
#min_support=0.02: Chỉ giữ lại các tập sản phẩm xuất hiện trong ít nhất 2% giao dịch.
#use_colnames=True: Sử dụng tên sản phẩm làm đầu ra thay vì chỉ số cột.

# Hiển thị các tập phổ biến
print(frequent_itemsets)


In [ ]:
from mlxtend.frequent_patterns import association_rules
# Tạo các quy tắc liên kết
rules = association_rules(frequent_itemsets, metric="lift" , min_threshold=1.0,num_itemsets = len(frequent_itemsets))
print("\nCác quy tắc liên kết:")
print(rules[['antecedents', 'consequents', 'support', 'confidence', 'lift']])

In [ ]:
# Chuyển đổi các itemset (frozenset) thành chuỗi không có "frozenset"
frequent_itemsets['itemsets'] = frequent_itemsets['itemsets'].apply(lambda x: ', '.join(list(x)))

# Vẽ biểu đồ số lượng itemset và support
plt.figure(figsize=(12, 6))

# Vẽ biểu đồ cột với tên các itemset đã được chuyển đổi thành chuỗi
plt.bar(frequent_itemsets['itemsets'], frequent_itemsets['support'], color='skyblue')

# Thêm tiêu đề và nhãn cho trục
plt.title('Biểu đồ các itemset phổ biến', fontsize=16)
plt.xlabel('Itemsets', fontsize=12)
plt.ylabel('Support', fontsize=12)

# Xoay trục x nếu cần thiết (nếu có nhiều itemset)
plt.xticks(rotation=90)

# Hiển thị biểu đồ
plt.show()

In [ ]:
# Đếm số lần xuất hiện của mỗi sản phẩm
item_counts = df['item_name'].value_counts()
# Chuyển thành DataFrame để dễ hiển thị
item_counts_df = item_counts.reset_index()
item_counts_df.columns = ['item_name', 'count']
print(item_counts_df)  

In [ ]:
print(frequent_itemsets.head())

In [ ]:
# Tạo ma trận đồng xuất hiện sản phẩm
co_occurrence_matrix = df_matrix.T.dot(df_matrix)
sns.heatmap(co_occurrence_matrix, cmap="Blues",linecolor='white',linewidth=1)
plt.title("Heatmap: Ma trận đồng xuất hiện sản phẩm")
plt.show()

In [ ]:
## Tìm các sản phẩm thường được mua cùng nhau
# Tạo bảng pivot để phân tích
#df.pivot_table(...): Tạo một bảng pivot từ DataFrame df với các thông tin sau:
#index='order_id': Mỗi dòng trong bảng pivot sẽ đại diện cho một order_id (mã đơn hàng).
#columns='item_name': Các cột sẽ là tên các sản phẩm (item_name).
#values='quantity': Các giá trị trong bảng là số lượng (quantity) của từng sản phẩm trong mỗi đơn hàng.
#aggfunc='sum': Nếu một đơn hàng mua cùng lúc nhiều sản phẩm giống nhau, các số lượng sẽ được cộng lại.
#fill_value=0: Nếu một sản phẩm không có trong một đơn hàng, giá trị sẽ là 0 (không mua sản phẩm đó trong đơn hàng đó).

basket = df.pivot_table(index='order_id', columns='item_name', values='quantity', aggfunc='sum', fill_value=0)

# Tính toán ma trận tương quan
basket_matrix = basket.corr()
print(basket_matrix)

In [ ]:
# Vẽ biểu đồ heatmap cho Ma trận tương quan
plt.figure(figsize=(12, 8))
sns.heatmap(basket_matrix, cmap='coolwarm', annot=False, cbar=True)
plt.title('Ma trận tương quan giữa các sản phẩm', fontsize=16)
plt.show()

In [ ]:
# Chuẩn bị dữ liệu
top_rules = rules.sort_values('lift', ascending=False).head(10)

# Tạo nhãn luật liên kết: Antecedents → Consequents
rule_labels = (
    top_rules['antecedents'].astype(str).str.replace("frozenset\(\{|\}\)", "", regex=True)
    + " → " +
    top_rules['consequents'].astype(str).str.replace("frozenset\(\{|\}\)", "", regex=True)
)

# Giá trị Lift, Confidence, và Support
lift_values = top_rules['lift']
confidence_values = top_rules['confidence']
support_values = top_rules['support']

# Định nghĩa vị trí cột
x = np.arange(len(rule_labels))  # Vị trí nhóm cột
width = 0.25  # Độ rộng mỗi cột

# Tạo biểu đồ cột nhóm
plt.figure(figsize=(14, 8))

# Cột Lift
plt.bar(x - width, lift_values, width, label='Lift', color='skyblue')

# Cột Confidence
plt.bar(x, confidence_values, width, label='Confidence', color='orange')

# Cột Support
plt.bar(x + width, support_values, width, label='Support', color='green')

# Tùy chỉnh biểu đồ
plt.xlabel('Luật liên kết (Antecedents → Consequents)', fontsize=12)
plt.ylabel('Giá trị', fontsize=12)
plt.title('So sánh Lift, Confidence và Support của Top 10 luật liên kết', fontsize=14)
plt.xticks(x, rule_labels, rotation=45, ha='right', fontsize=10)
plt.legend()  # Hiển thị chú thích
plt.tight_layout()  # Điều chỉnh bố cục để không bị tràn nội dung
plt.grid(axis='y', linestyle='--', alpha=0.6)

# Hiển thị biểu đồ
plt.show()

In [ ]:
import matplotlib.pyplot as plt

# Lấy top 10 luật liên kết theo lift
top_rules = rules.sort_values('lift', ascending=False).head(10)

# Vẽ biểu đồ Lift
plt.figure(figsize=(10, 6))
plt.barh(
    top_rules['antecedents'].astype(str).str.replace("frozenset\(\{|\}\)", "", regex=True),
    top_rules['lift'], 
    color='skyblue'
)
plt.xlabel('Lift')
plt.ylabel('Luật liên kết')
plt.title('Top 10 luật liên kết theo Lift')
plt.show()

# Vẽ biểu đồ Confidence
plt.figure(figsize=(10, 6))
plt.barh(
    top_rules['antecedents'].astype(str).str.replace("frozenset\(\{|\}\)", "", regex=True),
    top_rules['confidence'], 
    color='orange'
)
plt.xlabel('Confidence')
plt.ylabel('Luật liên kết')
plt.title('Top 10 luật liên kết theo Confidence')
plt.show()

# Vẽ biểu đồ Support
plt.figure(figsize=(10, 6))
plt.barh(
    top_rules['antecedents'].astype(str).str.replace("frozenset\(\{|\}\)", "", regex=True),
    top_rules['support'], 
    color='green'
)
plt.xlabel('Support')
plt.ylabel('Luật liên kết')
plt.title('Top 10 luật liên kết theo Support')
plt.show()

In [ ]:
# Tính tổng số lượng và doanh thu cho từng sản phẩm
product_summary = df.groupby('item_name').agg({'quantity': 'sum', 'item_price': 'sum'}).reset_index()
product_summary.columns = ['item_name', 'total_quantity', 'total_revenue']
print(product_summary.sort_values(by='total_revenue', ascending=False))

In [ ]:
# Vẽ biểu đồ
plt.figure(figsize=(12, 6))

# Top 10 sản phẩm theo doanh thu
top_10_revenue = product_summary.sort_values(by='total_revenue', ascending=False).head(10)

# Biểu đồ doanh thu
plt.subplot(1, 2, 1)
sns.barplot(x='total_revenue', y='item_name', data=top_10_revenue, palette='viridis')
plt.title('Top 10 sản phẩm theo doanh thu')
plt.xlabel('Doanh thu ($)')
plt.ylabel('Tên sản phẩm')

# Top 10 sản phẩm theo số lượng
top_10_quantity = product_summary.sort_values(by='total_quantity', ascending=False).head(10)

# Biểu đồ số lượng
plt.subplot(1, 2, 2)
sns.barplot(x='total_quantity', y='item_name', data=top_10_quantity, palette='magma')
plt.title('Top 10 sản phẩm theo số lượng')
plt.xlabel('Số lượng')
plt.ylabel('Tên sản phẩm')

plt.tight_layout()
plt.show()

In [ ]:
# Mã hóa giao dịch thành dạng ma trận
te = TransactionEncoder()
transaction_matrix = te.fit_transform(transactions)
transaction_df = pd.DataFrame(transaction_matrix, columns=te.columns_)

## THEO DOANH THU

In [ ]:
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE


In [ ]:
# Tính VIF cho từng đặc trưng
from statsmodels.stats.outliers_influence import variance_inflation_factor
vif_data = pd.DataFrame()
vif_data["Feature"] = transaction_df.columns
vif_data["VIF"] = [variance_inflation_factor(transaction_df.values, i) for i in range(transaction_df.shape[1])]

# In kết quả
print(vif_data) 
# VIF<5: không có hiện tượng đa cộng tuyến ->KHÔNG LOẠI BỎ BIẾN NÀO KHI CÓ HỆ SỐ ÂM

In [ ]:
transaction_df = transaction_df.astype(int)

In [ ]:
transaction_df.values

In [ ]:
# Tạo dữ liệu dự đoán
target_product = "Chicken Bowl"

# Đặt cột mục tiêu
transaction_df['target'] = transaction_df[target_product].astype(int)

# Loại bỏ sản phẩm mục tiêu khỏi đặc trưng dự đoán
X = transaction_df.drop(columns=['target', target_product])
y = transaction_df['target']

# X: Chứa các đặc trưng (features) dùng để huấn luyện mô hình. Tất cả các cột ngoài target và target_product đều được giữ lại.
# y: Là cột mục tiêu, chứa giá trị target mà mô hình sẽ dự đoán.
# X.astype(int): Chuyển các giá trị trong DataFrame X thành kiểu số nguyên (int), để phù hợp với yêu cầu của các mô hình học máy.
# y.astype(int): Đảm bảo cột y (cột mục tiêu) cũng có kiểu số nguyên
X = X.astype(int)
y = y.astype(int)


# Áp dụng SMOTE để tăng dữ liệu lớp 1
# SMOTE: Là một kỹ thuật tăng cường dữ liệu cho các lớp không cân bằng. Trong bài toán này, nếu lớp "1" (có sản phẩm mục tiêu) ít hơn lớp "0" (không có sản phẩm mục tiêu),SMOTE sẽ tạo ra các mẫu dữ liệu giả (synthetic samples) cho lớp "1" để làm cân bằng dữ liệu.
# fit_resample(X, y): Áp dụng SMOTE để tạo ra bộ dữ liệu mới (X_resampled, y_resampled) với số lượng mẫu của các lớp cân bằng hơn.
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

# Tách dữ liệu train-test
#train_test_split(...): Tách dữ liệu thành hai phần: một phần để huấn luyện mô hình và một phần để kiểm tra mô hình.
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

# Huấn luyện Logistic Regression
# LogisticRegression(max_iter=1000, random_state=42): Tạo một mô hình Logistic Regression với tối đa 1000 vòng lặp huấn luyện (max_iter) để đảm bảo hội tụ, và một hạt giống ngẫu nhiên (random_state=42) để tái tạo kết quả.
# model.fit(X_train, y_train): Huấn luyện mô hình trên dữ liệu huấn luyện (X_train và y_train).
model = LogisticRegression(max_iter=1000, random_state=42)
model.fit(X_train, y_train)

# Dự đoán và đánh giá
y_pred = model.predict(X_test)
print("Đánh giá mô hình:")
print(classification_report(y_test, y_pred))
print(f"Độ chính xác: {accuracy_score(y_test, y_pred)}")

# Trực quan hóa: Tầm quan trọng của các đặc trưng
coefficients = pd.Series(model.coef_[0], index=X.columns).sort_values(ascending=False)

plt.figure(figsize=(10, 6))
sns.barplot(x=coefficients[:10], y=coefficients[:10].index, palette="viridis")
plt.title("Top 10 món ăn quan trọng nhất trong dự đoán (Chicken Bowl)", fontsize=14)
plt.xlabel("Hệ số hồi quy", fontsize=12)
plt.ylabel("Tên sản phẩm", fontsize=12)
plt.grid(axis='x', linestyle='--', alpha=0.6)
plt.show()

In [ ]:
# Tạo dữ liệu dự đoán
target_product = "Chicken Burrito"

# Đặt cột mục tiêu
transaction_df['target'] = transaction_df[target_product].astype(int)

# Loại bỏ sản phẩm mục tiêu khỏi đặc trưng dự đoán
X = transaction_df.drop(columns=['target', target_product])
y = transaction_df['target']

X = X.astype(int)
y = y.astype(int)

# Áp dụng SMOTE để tăng dữ liệu lớp 1
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

# Tách dữ liệu train-test
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

# Huấn luyện Logistic Regression
model = LogisticRegression(max_iter=1000, random_state=42)
model.fit(X_train, y_train)

# Dự đoán và đánh giá
y_pred = model.predict(X_test)
print("Đánh giá mô hình:")
print(classification_report(y_test, y_pred))
print(f"Độ chính xác: {accuracy_score(y_test, y_pred)}")

In [ ]:
# Trực quan hóa: Tầm quan trọng của các đặc trưng
coefficients = pd.Series(model.coef_[0], index=X.columns).sort_values(ascending=False)

plt.figure(figsize=(10, 6))
sns.barplot(x=coefficients[:10], y=coefficients[:10].index, palette="viridis")
plt.title("Top 10 món ăn quan trọng nhất trong dự đoán (Chicken Burrito)", fontsize=14)
plt.xlabel("Hệ số hồi quy", fontsize=12)
plt.ylabel("Tên sản phẩm", fontsize=12)
plt.grid(axis='x', linestyle='--', alpha=0.6)
plt.show()

In [ ]:
# Tạo dữ liệu dự đoán
target_product = "Steak Bowl"

# Đặt cột mục tiêu
transaction_df['target'] = transaction_df[target_product].astype(int)

# Loại bỏ sản phẩm mục tiêu khỏi đặc trưng dự đoán
X = transaction_df.drop(columns=['target', target_product])
y = transaction_df['target']

X = X.astype(int)
y = y.astype(int)

# Áp dụng SMOTE để tăng dữ liệu lớp 1
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

# Tách dữ liệu train-test
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

# Huấn luyện Logistic Regression
model = LogisticRegression(max_iter=1000, random_state=42)
model.fit(X_train, y_train)

# Dự đoán và đánh giá
y_pred = model.predict(X_test)
print("Đánh giá mô hình:")
print(classification_report(y_test, y_pred))
print(f"Độ chính xác: {accuracy_score(y_test, y_pred)}")

In [ ]:
# Trực quan hóa: Tầm quan trọng của các đặc trưng
coefficients = pd.Series(model.coef_[0], index=X.columns).sort_values(ascending=False)

plt.figure(figsize=(10, 6))
sns.barplot(x=coefficients[:10], y=coefficients[:10].index, palette="viridis")
plt.title("Top 10 món ăn quan trọng nhất trong dự đoán (Steak Bowl)", fontsize=14)
plt.xlabel("Hệ số hồi quy", fontsize=12)
plt.ylabel("Tên sản phẩm", fontsize=12)
plt.grid(axis='x', linestyle='--', alpha=0.6)
plt.show()

In [ ]:
# Tạo dữ liệu dự đoán
target_product = "Chips and Guacamole"

# Đặt cột mục tiêu
transaction_df['target'] = transaction_df[target_product].astype(int)

# Loại bỏ sản phẩm mục tiêu khỏi đặc trưng dự đoán
X = transaction_df.drop(columns=['target', target_product])
y = transaction_df['target']

X = X.astype(int)
y = y.astype(int)

# Áp dụng SMOTE để tăng dữ liệu lớp 1
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

# Tách dữ liệu train-test
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

# Huấn luyện Logistic Regression
model = LogisticRegression(max_iter=1000, random_state=42)
model.fit(X_train, y_train)

# Dự đoán và đánh giá
y_pred = model.predict(X_test)
print("Đánh giá mô hình:")
print(classification_report(y_test, y_pred))
print(f"Độ chính xác: {accuracy_score(y_test, y_pred)}")

In [ ]:
# Trực quan hóa: Tầm quan trọng của các đặc trưng
coefficients = pd.Series(model.coef_[0], index=X.columns).sort_values(ascending=False)

plt.figure(figsize=(10, 6))
sns.barplot(x=coefficients[:10], y=coefficients[:10].index, palette="viridis")
plt.title("Top 10 món ăn quan trọng nhất trong dự đoán (Chips and Guacamole)", fontsize=14)
plt.xlabel("Hệ số hồi quy", fontsize=12)
plt.ylabel("Tên sản phẩm", fontsize=12)
plt.grid(axis='x', linestyle='--', alpha=0.6)
plt.show()

In [ ]:
# Tạo dữ liệu dự đoán
target_product = "Veggie Burrito"

# Đặt cột mục tiêu
transaction_df['target'] = transaction_df[target_product].astype(int)

# Loại bỏ sản phẩm mục tiêu khỏi đặc trưng dự đoán
X = transaction_df.drop(columns=['target', target_product])
y = transaction_df['target']

X = X.astype(int)
y = y.astype(int)

# Áp dụng SMOTE để tăng dữ liệu lớp 1
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

# Tách dữ liệu train-test
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

# Huấn luyện Logistic Regression
model = LogisticRegression(max_iter=1000, random_state=42)
model.fit(X_train, y_train)

# Dự đoán và đánh giá
y_pred = model.predict(X_test)
print("Đánh giá mô hình:")
print(classification_report(y_test, y_pred))
print(f"Độ chính xác: {accuracy_score(y_test, y_pred)}")

# Trực quan hóa: Tầm quan trọng của các đặc trưng
coefficients = pd.Series(model.coef_[0], index=X.columns).sort_values(ascending=False)

plt.figure(figsize=(10, 6))
sns.barplot(x=coefficients[:10], y=coefficients[:10].index, palette="viridis")
plt.title("Top 10 đặc trưng quan trọng nhất trong dự đoán (Veggie Burrito)", fontsize=14)
plt.xlabel("Hệ số hồi quy", fontsize=12)
plt.ylabel("Tên sản phẩm", fontsize=12)
plt.grid(axis='x', linestyle='--', alpha=0.6)
plt.show()

In [ ]:
# Tạo dữ liệu dự đoán
target_product = "Barbacoa Burrito"

# Đặt cột mục tiêu
transaction_df['target'] = transaction_df[target_product].astype(int)

# Loại bỏ sản phẩm mục tiêu khỏi đặc trưng dự đoán
X = transaction_df.drop(columns=['target', target_product])
y = transaction_df['target']

X = X.astype(int)
y = y.astype(int)

# Áp dụng SMOTE để tăng dữ liệu lớp 1
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

# Tách dữ liệu train-test
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

# Huấn luyện Logistic Regression
model = LogisticRegression(max_iter=1000, random_state=42)
model.fit(X_train, y_train)

# Dự đoán và đánh giá
y_pred = model.predict(X_test)
print("Đánh giá mô hình:")
print(classification_report(y_test, y_pred))
print(f"Độ chính xác: {accuracy_score(y_test, y_pred)}")

# Trực quan hóa: Tầm quan trọng của các đặc trưng
coefficients = pd.Series(model.coef_[0], index=X.columns).sort_values(ascending=False)

plt.figure(figsize=(10, 6))
sns.barplot(x=coefficients[:10], y=coefficients[:10].index, palette="viridis")
plt.title("Top 10 đặc trưng quan trọng nhất trong dự đoán (Barbacoa Burrito)", fontsize=14)
plt.xlabel("Hệ số hồi quy", fontsize=12)
plt.ylabel("Tên sản phẩm", fontsize=12)
plt.grid(axis='x', linestyle='--', alpha=0.6)
plt.show()

In [ ]:
# Tạo dữ liệu dự đoán
target_product = "Veggie Bowl"

# Đặt cột mục tiêu
transaction_df['target'] = transaction_df[target_product].astype(int)

# Loại bỏ sản phẩm mục tiêu khỏi đặc trưng dự đoán
X = transaction_df.drop(columns=['target', target_product])
y = transaction_df['target']

X = X.astype(int)
y = y.astype(int)

# Áp dụng SMOTE để tăng dữ liệu lớp 1
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

# Tách dữ liệu train-test
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

# Huấn luyện Logistic Regression
model = LogisticRegression(max_iter=1000, random_state=42)
model.fit(X_train, y_train)

# Dự đoán và đánh giá
y_pred = model.predict(X_test)
print("Đánh giá mô hình:")
print(classification_report(y_test, y_pred))
print(f"Độ chính xác: {accuracy_score(y_test, y_pred)}")

In [ ]:
# Trực quan hóa: Tầm quan trọng của các đặc trưng
coefficients = pd.Series(model.coef_[0], index=X.columns).sort_values(ascending=False)

plt.figure(figsize=(10, 6))
sns.barplot(x=coefficients[:10], y=coefficients[:10].index, palette="viridis")
plt.title("Top 10 món ăn quan trọng nhất trong dự đoán (Veggie Bowl)", fontsize=14)
plt.xlabel("Hệ số hồi quy", fontsize=12)
plt.ylabel("Tên sản phẩm", fontsize=12)
plt.grid(axis='x', linestyle='--', alpha=0.6)
plt.show()

## THEO SỐ LƯỢNG

In [ ]:
# Tạo dữ liệu dự đoán
target_product = "Canned Soft Drink"

# Đặt cột mục tiêu
transaction_df['target'] = transaction_df[target_product].astype(int)

# Loại bỏ sản phẩm mục tiêu khỏi đặc trưng dự đoán
X = transaction_df.drop(columns=['target', target_product])
y = transaction_df['target']

X = X.astype(int)
y = y.astype(int)

# Áp dụng SMOTE để tăng dữ liệu lớp 1
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

# Tách dữ liệu train-test
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

# Huấn luyện Logistic Regression
model = LogisticRegression(max_iter=1000, random_state=42)
model.fit(X_train, y_train)

# Dự đoán và đánh giá
y_pred = model.predict(X_test)
print("Đánh giá mô hình:")
print(classification_report(y_test, y_pred))
print(f"Độ chính xác: {accuracy_score(y_test, y_pred)}")

In [ ]:
# Trực quan hóa: Tầm quan trọng của các đặc trưng
coefficients = pd.Series(model.coef_[0], index=X.columns).sort_values(ascending=False)

plt.figure(figsize=(10, 6))
sns.barplot(x=coefficients[:10], y=coefficients[:10].index, palette="viridis")
plt.title("Top 10 món ăn quan trọng nhất trong dự đoán (Canned Soft Drink)", fontsize=14)
plt.xlabel("Hệ số hồi quy", fontsize=12)
plt.ylabel("Tên sản phẩm", fontsize=12)
plt.grid(axis='x', linestyle='--', alpha=0.6)
plt.show()

In [ ]:
# Tạo dữ liệu dự đoán
target_product = "Steak Burrito"

# Đặt cột mục tiêu
transaction_df['target'] = transaction_df[target_product].astype(int)

# Loại bỏ sản phẩm mục tiêu khỏi đặc trưng dự đoán
X = transaction_df.drop(columns=['target', target_product])
y = transaction_df['target']

X = X.astype(int)
y = y.astype(int)

# Áp dụng SMOTE để tăng dữ liệu lớp 1
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

# Tách dữ liệu train-test
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

# Huấn luyện Logistic Regression
model = LogisticRegression(max_iter=1000, random_state=42)
model.fit(X_train, y_train)

# Dự đoán và đánh giá
y_pred = model.predict(X_test)
print("Đánh giá mô hình:")
print(classification_report(y_test, y_pred))
print(f"Độ chính xác: {accuracy_score(y_test, y_pred)}")

In [ ]:
# Trực quan hóa: Tầm quan trọng của các đặc trưng
coefficients = pd.Series(model.coef_[0], index=X.columns).sort_values(ascending=False)

plt.figure(figsize=(10, 6))
sns.barplot(x=coefficients[:10], y=coefficients[:10].index, palette="viridis")
plt.title("Top 10 món ăn quan trọng nhất trong dự đoán (Steak Burrito)", fontsize=14)
plt.xlabel("Hệ số hồi quy", fontsize=12)
plt.ylabel("Tên sản phẩm", fontsize=12)
plt.grid(axis='x', linestyle='--', alpha=0.6)
plt.show()

In [ ]:
# Tạo dữ liệu dự đoán
target_product = "Bottled Water"

# Đặt cột mục tiêu
transaction_df['target'] = transaction_df[target_product].astype(int)

# Loại bỏ sản phẩm mục tiêu khỏi đặc trưng dự đoán
X = transaction_df.drop(columns=['target', target_product])
y = transaction_df['target']

X = X.astype(int)
y = y.astype(int)

# Áp dụng SMOTE để tăng dữ liệu lớp 1
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

# Tách dữ liệu train-test
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

# Huấn luyện Logistic Regression
model = LogisticRegression(max_iter=1000, random_state=42)
model.fit(X_train, y_train)

# Dự đoán và đánh giá
y_pred = model.predict(X_test)
print("Đánh giá mô hình:")
print(classification_report(y_test, y_pred))
print(f"Độ chính xác: {accuracy_score(y_test, y_pred)}")



In [ ]:
# Trực quan hóa: Tầm quan trọng của các đặc trưng
coefficients = pd.Series(model.coef_[0], index=X.columns).sort_values(ascending=False)

plt.figure(figsize=(10, 6))
sns.barplot(x=coefficients[:10], y=coefficients[:10].index, palette="viridis")
plt.title("Top 10 món ăn quan trọng nhất trong dự đoán (Bottled Water)", fontsize=14)
plt.xlabel("Hệ số hồi quy", fontsize=12)
plt.ylabel("Tên sản phẩm", fontsize=12)
plt.grid(axis='x', linestyle='--', alpha=0.6)
plt.show()

In [ ]:
# Tạo dữ liệu dự đoán
target_product = "Canned Soda"

# Đặt cột mục tiêu
transaction_df['target'] = transaction_df[target_product].astype(int)

# Loại bỏ sản phẩm mục tiêu khỏi đặc trưng dự đoán
X = transaction_df.drop(columns=['target', target_product])
y = transaction_df['target']

X = X.astype(int)
y = y.astype(int)

# Áp dụng SMOTE để tăng dữ liệu lớp 1
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

# Tách dữ liệu train-test
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

# Huấn luyện Logistic Regression
model = LogisticRegression(max_iter=1000, random_state=42)
model.fit(X_train, y_train)

# Dự đoán và đánh giá
y_pred = model.predict(X_test)
print("Đánh giá mô hình:")
print(classification_report(y_test, y_pred))
print(f"Độ chính xác: {accuracy_score(y_test, y_pred)}")



In [ ]:
# Trực quan hóa: Tầm quan trọng của các đặc trưng
coefficients = pd.Series(model.coef_[0], index=X.columns).sort_values(ascending=False)

plt.figure(figsize=(10, 6))
sns.barplot(x=coefficients[:10], y=coefficients[:10].index, palette="viridis")
plt.title("Top 10 món ăn quan trọng nhất trong dự đoán (Canned Soda)", fontsize=14)
plt.xlabel("Hệ số hồi quy", fontsize=12)
plt.ylabel("Tên sản phẩm", fontsize=12)
plt.grid(axis='x', linestyle='--', alpha=0.6)
plt.show()

In [ ]:
# Tạo dữ liệu dự đoán
target_product = "Chips"

# Đặt cột mục tiêu
transaction_df['target'] = transaction_df[target_product].astype(int)

# Loại bỏ sản phẩm mục tiêu khỏi đặc trưng dự đoán
X = transaction_df.drop(columns=['target', target_product])
y = transaction_df['target']

X = X.astype(int)
y = y.astype(int)

# Áp dụng SMOTE để tăng dữ liệu lớp 1
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

# Tách dữ liệu train-test
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

# Huấn luyện Logistic Regression
model = LogisticRegression(max_iter=1000, random_state=42)
model.fit(X_train, y_train)

# Dự đoán và đánh giá
y_pred = model.predict(X_test)
print("Đánh giá mô hình:")
print(classification_report(y_test, y_pred))
print(f"Độ chính xác: {accuracy_score(y_test, y_pred)}")

In [ ]:
# Trực quan hóa: Tầm quan trọng của các đặc trưng
coefficients = pd.Series(model.coef_[0], index=X.columns).sort_values(ascending=False)

plt.figure(figsize=(10, 6))
sns.barplot(x=coefficients[:10], y=coefficients[:10].index, palette="viridis")
plt.title("Top 10 món ăn quan trọng nhất trong dự đoán (Chips)", fontsize=14)
plt.xlabel("Hệ số hồi quy", fontsize=12)
plt.ylabel("Tên sản phẩm", fontsize=12)
plt.grid(axis='x', linestyle='--', alpha=0.6)
plt.show()

In [ ]:
# Tạo dữ liệu dự đoán
target_product = "Chips and Fresh Tomato Salsa"

# Đặt cột mục tiêu
transaction_df['target'] = transaction_df[target_product].astype(int)

# Loại bỏ sản phẩm mục tiêu khỏi đặc trưng dự đoán
X = transaction_df.drop(columns=['target', target_product])
y = transaction_df['target']

X = X.astype(int)
y = y.astype(int)

# Áp dụng SMOTE để tăng dữ liệu lớp 1
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

# Tách dữ liệu train-test
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

# Huấn luyện Logistic Regression
model = LogisticRegression(max_iter=1000, random_state=42)
model.fit(X_train, y_train)

# Dự đoán và đánh giá
y_pred = model.predict(X_test)
print("Đánh giá mô hình:")
print(classification_report(y_test, y_pred))
print(f"Độ chính xác: {accuracy_score(y_test, y_pred)}")

In [ ]:
# Trực quan hóa: Tầm quan trọng của các đặc trưng
coefficients = pd.Series(model.coef_[0], index=X.columns).sort_values(ascending=False)

plt.figure(figsize=(10, 6))
sns.barplot(x=coefficients[:10], y=coefficients[:10].index, palette="viridis")
plt.title("Top 10 món ăn quan trọng nhất trong dự đoán (Chips and Fresh Tomato Salsa)", fontsize=14)
plt.xlabel("Hệ số hồi quy", fontsize=12)
plt.ylabel("Tên sản phẩm", fontsize=12)
plt.grid(axis='x', linestyle='--', alpha=0.6)
plt.show()

In [ ]:
# Tạo dữ liệu dự đoán
target_product = "Chicken Salad Bowl"

# Đặt cột mục tiêu
transaction_df['target'] = transaction_df[target_product].astype(int)

# Loại bỏ sản phẩm mục tiêu khỏi đặc trưng dự đoán
X = transaction_df.drop(columns=['target', target_product])
y = transaction_df['target']

X = X.astype(int)
y = y.astype(int)

# Áp dụng SMOTE để tăng dữ liệu lớp 1
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

# Tách dữ liệu train-test
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

# Huấn luyện Logistic Regression
model = LogisticRegression(max_iter=1000, random_state=42)
model.fit(X_train, y_train)

# Dự đoán và đánh giá
y_pred = model.predict(X_test)
print("Đánh giá mô hình:")
print(classification_report(y_test, y_pred))
print(f"Độ chính xác: {accuracy_score(y_test, y_pred)}")

In [ ]:
# Trực quan hóa: Tầm quan trọng của các đặc trưng
coefficients = pd.Series(model.coef_[0], index=X.columns).sort_values(ascending=False)

plt.figure(figsize=(10, 6))
sns.barplot(x=coefficients[:10], y=coefficients[:10].index, palette="viridis")
plt.title("Top 10 món ăn quan trọng nhất trong dự đoán (Chicken Salad Bowl)", fontsize=14)
plt.xlabel("Hệ số hồi quy", fontsize=12)
plt.ylabel("Tên sản phẩm", fontsize=12)
plt.grid(axis='x', linestyle='--', alpha=0.6)
plt.show()

In [ ]:
# Tạo dữ liệu dự đoán
target_product = "Chicken Soft Tacos"

# Đặt cột mục tiêu
transaction_df['target'] = transaction_df[target_product].astype(int)

# Loại bỏ sản phẩm mục tiêu khỏi đặc trưng dự đoán
X = transaction_df.drop(columns=['target', target_product])
y = transaction_df['target']

X = X.astype(int)
y = y.astype(int)

# Áp dụng SMOTE để tăng dữ liệu lớp 1
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

# Tách dữ liệu train-test
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

# Huấn luyện Logistic Regression
model = LogisticRegression(max_iter=1000, random_state=42)
model.fit(X_train, y_train)

# Dự đoán và đánh giá
y_pred = model.predict(X_test)
print("Đánh giá mô hình:")
print(classification_report(y_test, y_pred))
print(f"Độ chính xác: {accuracy_score(y_test, y_pred)}")

In [ ]:
# Trực quan hóa: Tầm quan trọng của các đặc trưng
coefficients = pd.Series(model.coef_[0], index=X.columns).sort_values(ascending=False)

plt.figure(figsize=(10, 6))
sns.barplot(x=coefficients[:10], y=coefficients[:10].index, palette="viridis")
plt.title("Top 10 món ăn quan trọng nhất trong dự đoán (Chicken Soft Tacos)", fontsize=14)
plt.xlabel("Hệ số hồi quy", fontsize=12)
plt.ylabel("Tên sản phẩm", fontsize=12)
plt.grid(axis='x', linestyle='--', alpha=0.6)
plt.show()